In [82]:
import pandas as pd 
import numpy as np
input_df = pd.read_csv('../data/train/input_2023_w01.csv')
output_df = pd.read_csv('../data/train/output_2023_w01.csv')
supplemental_df = pd.read_csv('../data/supplementary_data.csv')
pd.set_option('display.max_columns', None)

df = pd.merge(input_df, supplemental_df, on=['game_id','play_id'], how='left')
print(df.head())




/var/folders/7d/j6bnzr6j1s9dc0n2tggc7syr0000gp/T/ipykernel_21454/1444866542.py:5: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  supplemental_df = pd.read_csv('../data/supplementary_data.csv')


      game_id  play_id  player_to_predict  nfl_id  frame_id play_direction  \
0  2023090700      101              False   54527         1          right   
1  2023090700      101              False   54527         2          right   
2  2023090700      101              False   54527         3          right   
3  2023090700      101              False   54527         4          right   
4  2023090700      101              False   54527         5          right   

   absolute_yardline_number player_name player_height  player_weight  \
0                        42  Bryan Cook           6-1            210   
1                        42  Bryan Cook           6-1            210   
2                        42  Bryan Cook           6-1            210   
3                        42  Bryan Cook           6-1            210   
4                        42  Bryan Cook           6-1            210   

  player_birth_date player_position player_side         player_role      x  \
0        1999-09-07 

In [83]:

# Filtering out Zone coverages
df_man =  df[~df['team_coverage_type'].str.contains("ZONE")]
print(df_man['team_coverage_type'].value_counts())


COVER_1_MAN    60120
COVER_0_MAN     7546
COVER_2_MAN     5605
Name: team_coverage_type, dtype: int64


In [ ]:
df_man['x_player_ball_dist'] = df_man['ball_land_x']-df_man['x']
df_man['y_player_ball_dist'] = df_man['ball_land_y']-df_man['y']
# Calculate Euclidean distance from player to ball landing location
df_man['euclidean_dist_to_ball'] = np.sqrt(df_man['x_player_ball_dist']**2 + df_man['y_player_ball_dist']**2)
#print(df_man.head())
#print(df_man[df_man['play_id'] == 2389][['player_name', 'player_position', 'possession_team', 'player_to_predict']])

#Filter out non predicted players


df_man_pred = df_man[(df_man['player_to_predict'] == True) & df_man['player_position'].isin(['CB', 'S', 'DB', 'FS', 'SS', 'WR'])]

print(df_man_pred.head())


In [ ]:
#Binary variable 1 = cut off route, 0 = stayed on WR
#If defender farther from line of scrimmage than WR at time of catch then 0 (stayed on WR)
#If defender closer to line of scrimmage than WR at time of catch then 1 (cut off route)
if df_man_pred['play_direction' == 'right']:
    df_man_pred[' '] = df_man_pred['x'] - df_man_pred['absolute_yardline_number']
if df_man_pred['play_direction' == 'left']:    
    df_man_pred['x'] - df_man_pred['absolute_yardline_number']


In [85]:


# # Remove all Offessive players except Wide Receivers
# df_man = df_man[(df_man['player_side'] == 'Defense') | (df_man['player_position'] == 'WR')]
# print(df_man['player_position'].value_counts())


# # First filter to only WRs and Defense
# df_man = df_man[(df_man['player_side'] == 'Defense') | (df_man['player_position'] == 'WR')]

# # Then group by player_side to find closest on each side
# min_dist_per_frame = df_man.groupby(['game_id', 'play_id', 'frame_id', 'player_side'])['euclidean_dist_to_ball'].transform('min')

# # Create boolean variable
# df_man['is_closest_to_ball_landing'] = df_man['euclidean_dist_to_ball'] == min_dist_per_frame  - USEFUL LATER     


In [86]:

# df_closest_receiver = df_man

# # Get the maximum frame_id for each play
# last_frame = df_closest_receiver.groupby(['game_id', 'play_id'])['frame_id'].transform('max')

# # Find which WR is closest on the last frame for each play
# last_frame_data = df_closest_receiver[df_closest_receiver['frame_id'] == last_frame].copy()

# # Get the nfl_id of the player that is closest on the last frame for each play AND side
# closest_player_on_last_frame = last_frame_data[
#     last_frame_data['is_closest_to_ball_landing']
# ][['game_id', 'play_id', 'nfl_id', 'player_side']]

# # Merge back to mark all frames of that player
# df_closest_receiver = df_closest_receiver.merge(
#     closest_player_on_last_frame.assign(will_be_closest_on_their_side=True),
#     on=['game_id', 'play_id', 'nfl_id'],
#     how='left'
# )

# # Fill NaN with False
# df_closest_receiver['will_be_closest_on_their_side'] = df_closest_receiver['will_be_closest_on_their_side'].fillna(False)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', 50)
# #print(df_closest_receiver[(df_closest_receiver['will_be_closest_wr'] == True) & (df_closest_receiver['player_name'] == 'A.J. Brown')])
# print(df_closest_receiver[df_closest_receiver['game_id'] == 2023090700][['player_name','player_position','possession_team', 'will_be_closest_on_their_side', 'is_closest_to_ball_landing']])




In [87]:
# #Filter to only include WR that will be closest
# df_final = df_closest_receiver[(df_closest_receiver['will_be_closest_on_their_side'] == True) | (df_closest_receiver['player_position'] != 'WR')]
# #print(df_final[['player_name', 'player_position', 'possession_team', 'will_be_closest_on_their_side', 'is_closest_to_ball_landing']])

# #Create boolean for if the Defensive player is closest to the WR that will be closest to the ball landing
# # First, get the WR who will be closest on the last frame for each play
# closest_wr_per_play = df_final[
#     (df_final['player_position'] == 'WR') & 
#     (df_final['will_be_closest_on_their_side'])
# ][['game_id', 'play_id', 'frame_id', 'nfl_id', 'x', 'y']].rename(
#     columns={'nfl_id': 'target_wr_id', 'x': 'wr_x', 'y': 'wr_y'}
# )

# # Merge to get WR location for each frame
# df_with_wr_location = df_final.merge(
#     closest_wr_per_play,
#     on=['game_id', 'play_id', 'frame_id'],
#     how='left',
#     suffixes=('', '_wr')
# )

# # Calculate distance from each player to the target WR (only where target WR exists)
# df_with_wr_location['dist_to_target_wr'] = np.sqrt(
#     (df_with_wr_location['x'] - df_with_wr_location['wr_x'])**2 + 
#     (df_with_wr_location['y'] - df_with_wr_location['wr_y'])**2
# )

# # For defensive players only, find who is closest to the target WR per frame
# # Use the original player_side column (no suffix)
# defense_mask = df_with_wr_location['player_position'].isin(['CB', 'FS', 'SS', 'ILB', 'MLB', 'OLB', 'S', 'DE', 'DT', 'NT'])
# defense_only = df_with_wr_location[defense_mask].copy()

# min_dist_to_wr = defense_only.groupby(['game_id', 'play_id', 'frame_id'])['dist_to_target_wr'].transform('min')
# defense_only['is_closest_defender_to_target_wr'] = defense_only['dist_to_target_wr'] == min_dist_to_wr

# # Merge back to the main dataframe
# df_final = df_with_wr_location.merge(
#     defense_only[['game_id', 'play_id', 'frame_id', 'nfl_id', 'is_closest_defender_to_target_wr']],
#     on=['game_id', 'play_id', 'frame_id', 'nfl_id'],
#     how='left'
# )

# # Fill NaN with False (for WRs and non-closest defenders)
# df_final['is_closest_defender_to_target_wr'] = df_final['is_closest_defender_to_target_wr'].fillna(False)

# #print(df_final[df_final['game_id'] == 2023090700][['player_name','player_position','possession_team', 'will_be_closest_on_their_side', 'is_closest_to_ball_landing', 'is_closest_defender_to_target_wr']])
# pd.set_option('display.max_colwidth', None)
# #print(df_final[df_final['play_description'].str.contains("intercept", case=False, na=False)][['play_description', 'play_id']])

# print(df_final[df_final['play_id'] == 2389][['player_name', 'player_position', 'possession_team', 'player_to_predict']])


